# Stixrude-Lithgow-Bertelloni pseudo-omnicomponent phase generation
Required Python packages/modules

In [ ]:
import numpy as np
from os import path
import pandas as pd
import scipy.optimize as opt
import scipy.linalg as lin 
import scipy as sp
import sys
import sympy as sym

import matplotlib.pyplot as plt

Required ENKI modules (ignore the error message from Rubicon running under Python 3.6+)

In [ ]:
from thermoengine import coder, core, phases, model, equilibrate

In [ ]:
def get_subsolidus_phases(database='Berman'):
    remove_phases = ['Liq','H2O']
    
    modelDB = model.Database(database)
    phases = modelDB.phases
    [phases.pop(phs) for phs in remove_phases]
        
    return phases
        
def system_energy_landscape(T, P, phases, TOL=1e-3):
    elem_comps = []
    phs_sym = []
    endmem_ids = []
    soln_phase_names = []
    soln_endmem_ids = []
    pure_endmem_ids = []
    
    mu = []
    ind = 0
    for phsnm in phases:
        phs = phases[phsnm]
        
        elem_comp = phs.props['element_comp']
        abbrev = phs.abbrev
        endmem_num = phs.endmember_num
        iendmem_ids = np.arange(endmem_num)
        
        if phs.phase_type=='pure':
            nelem = np.sum(elem_comp)
            mu += [phs.gibbs_energy(T, P)/nelem]
            pure_endmem_ids.append(ind)
            # print(nelem)
        else:
            soln_phase_names.append(phsnm)
            nelem = np.sum(elem_comp,axis=1)
            soln_endmem_ids.append(ind+iendmem_ids)
            # print(nelem)
            for i in iendmem_ids:
                imol = np.eye(phs.endmember_num)[i]
                mu += [phs.gibbs_energy(T, P, mol=imol,deriv={"dmol":1})[0,i]/nelem[i]]
                # print(nelem[i])
                
        endmem_ids.extend(iendmem_ids)
        phs_sym.extend(list(np.tile(abbrev,endmem_num)))
        # print(elem_comp)
        
        elem_comps.extend(elem_comp)
        # print(elem_comp)
        # print(phs)
        ind += len(iendmem_ids)
        
    soln_phases = [phases[iname] for iname in soln_phase_names]
        
    elem_comps = np.vstack(elem_comps)
    
    natoms = np.sum(elem_comps,axis=1)
    elem_comps = elem_comps/natoms[:,np.newaxis]
    
    elem_mask = ~np.all(elem_comps<TOL, axis=0)
    # soln_phase_mask = np.array(soln_phase_mask)
    # soln_phase_names = np.array(soln_phase_names)
    
    elem_comps = elem_comps[:, elem_mask]
    mu = np.array(mu)
    endmem_ids = np.array(endmem_ids)
    phs_sym = np.array(phs_sym)
    pure_endmem_ids = np.array(pure_endmem_ids)
    
    sys_elems = core.chem.PERIODIC_ORDER[elem_mask]
    # return phs_sym, soln_phases, endmem_ids, mu, elem_comps, sys_elems
    return phs_sym, soln_phases, soln_endmem_ids, pure_endmem_ids, endmem_ids, mu, elem_comps, sys_elems

def prune_polymorphs(phs_sym, endmem_ids, mu, elem_comps, decimals=4):
    elem_round_comps = np.round(elem_comps, decimals=decimals)
        # Drop identical comps
    elem_comps_uniq = np.unique(elem_round_comps, axis=0)
    
    # uniq_num = elem_comps_uniq.shape[0]
    mu_uniq = []
    phs_sym_uniq = []
    endmem_ids_uniq = []
    for elem_comp in elem_comps_uniq:
        is_equiv_comp = np.all(elem_round_comps == elem_comp[np.newaxis,:], axis=1)
        equiv_ind = np.where(is_equiv_comp)[0]
        min_ind = equiv_ind[np.argsort(mu[equiv_ind])[0]]
        min_mu = mu[min_ind]
        assert np.all(min_mu <= mu[equiv_ind]), 'fail'
        
        mu_uniq.append(min_mu)
        phs_sym_uniq.append(phs_sym[min_ind])
        endmem_ids_uniq.append(endmem_ids[min_ind])
        
    mu_uniq = np.array(mu_uniq)
    phs_sym_uniq = np.array(phs_sym_uniq)
    elem_comps_uniq = np.array(elem_comps_uniq)
    
    return phs_sym_uniq, endmem_ids_uniq, mu_uniq, elem_comps_uniq



### T,P, parameters and options for pseudo-phase generation

In [ ]:
T = 1300.0                  # K
P = 20000.0                 # bars
P = 300000.0                 # bars
# T = 1500.0                  # K
# P = 40000.0                 # bars

In [ ]:
database='Berman'
database='Stixrude'

In [ ]:
test_endmember_code = False # output tests to validate solution endmember code generation
test_solution_code = False  # output tests to validate solution code generation
offset_value = 0.0          # Offset penality (in J) to destabilize pseudo-omnicomponent phase 4000
use_oxides_as_basis = False # Construct the pseudo-phase using oxides as components (False == elements)

In [ ]:
phases = get_subsolidus_phases(database=database)
# ADD extra phases (e.g. carbonates as needed here)

In [ ]:
# phs_sym, soln_phases, endmem_ids, mu, elem_comps, sys_elems = system_energy_landscape(T, P, phases)
# phs_sym, soln_phase_names, endmem_ids, mu, elem_comps, sys_elems = system_energy_landscape(T, P, phases)

phs_sym, soln_phases, soln_endmem_ids, pure_endmem_ids, endmem_ids, mu, elem_comps, sys_elems = (
    system_energy_landscape(T, P, phases))
# display(phs_sym, endmem_ids, mu, elem_comps, sys_elems)
phs_sym_uniq, endmem_ids_uniq, mu_uniq, elem_comps_uniq = (
    prune_polymorphs(phs_sym, endmem_ids, mu, elem_comps))
Nelems = len(sys_elems)
Npts = mu_uniq.size

In [ ]:
pure_mu = mu[pure_endmem_ids]
pure_mu

In [ ]:
soln_phases
soln_endmem_ids


all_soln_endmem_ids = np.hstack(soln_endmem_ids)

In [ ]:
# soln_phases = [phases[iname] for iname in soln_phase_names]
# soln_phases

In [ ]:
phs_sym

In [ ]:
elem_comps_uniq

In [ ]:
mu_uniq

In [ ]:
sys_elems

In [ ]:
comps = elem_comps_uniq
mu = mu_uniq

In [ ]:
comps

In [ ]:
def min_energy_assemblage(bulk_comp, comp, mu, TOLmu=10, TOL=1e-5, mu_init=None):
    xy = np.hstack((comp, mu[:,np.newaxis]))
    
    if mu_init is None:
        mu_init = np.mean(mu)
        
    xy_bulk = np.hstack((bulk_comp, mu_init))
    
    wt0, rnorm0 = opt.nnls(xy.T, xy_bulk)
        
    # print('rnorm',rnorm0)
    
    
    def fun(mu, shift=0):
        xy_bulk[-1] = mu
        wt, rnorm = opt.nnls(xy.T, xy_bulk)
        return rnorm-shift
    
    
    delmu = .1
    if rnorm0==0:
        shift_dir = -1
        soln_found = True
    else:
        output = opt.minimize_scalar(fun, bounds=[np.min(mu), np.max(mu)])
        xy_bulk[-1] = output['x']
        wt0, rnorm0 = opt.nnls(xy.T, xy_bulk)
        shift_dir = -1
        
    mu_prev=xy_bulk[-1]
    rnorm=rnorm0
    
    while True:
        mu_prev = xy_bulk[-1]
        rnorm_prev = rnorm
        
        xy_bulk[-1] += shift_dir*delmu
        wt, rnorm = opt.nnls(xy.T, xy_bulk)
        delmu *= 2
        
        # print(shift_dir, rnorm)
        if ((shift_dir==+1)&(rnorm>rnorm_prev)) or ((shift_dir==-1)&(rnorm>0)):
            break
            
        
    fun_fit = lambda mu, TOL=TOL: fun(mu, shift=TOL)
    if rnorm > TOL:
        mu_bulk = opt.brentq(fun_fit, mu_prev, xy_bulk[-1], xtol=TOLmu)
        xy_bulk[-1] = mu_bulk
        wt, rnorm = opt.nnls(xy.T, xy_bulk)
        
    mu_bulk = xy_bulk[-1]
    wt_bulk = wt
        
        
    ind_assem = np.where(wt_bulk>0)[0]
    return wt_bulk, mu_bulk, ind_assem 


# Define bulk composition

In [ ]:
wt = np.random.rand(elem_comps_uniq.shape[0])
wt = wt/np.sum(wt)
bulk_comp = np.dot(wt, elem_comps_uniq)

# Get minimum energy assemblage of pure phases

In [ ]:
wt_bulk, mu_bulk, ind_assem = min_energy_assemblage(
    bulk_comp, elem_comps_uniq, mu_uniq, TOLmu=10)
# print(len(ind_assem))

np.sum(wt_bulk)
np.sum(wt_bulk>0)

In [ ]:
mu_bulk

In [ ]:
np.dot(wt_bulk, elem_comps_uniq)

In [ ]:
elem_comps_assem0 = elem_comps_uniq[wt_bulk>0,:]
mu_assem0 = mu_uniq[wt_bulk>0]

output = np.linalg.lstsq(elem_comps_assem0, mu_assem0, rcond=None)
mu_elem_local = output[0]

In [ ]:
# elem_comps_assem0

In [ ]:
wt_bulk_assem0 = wt_bulk[wt_bulk>0]
print(np.dot(mu_elem_local, bulk_comp))
print(mu_bulk)

In [ ]:

Nsoln_tot = np.sum([ids.size for ids in soln_endmem_ids])
soln_mu = np.zeros(Nsoln_tot)


In [ ]:

for phs, ids in zip(soln_phases, soln_endmem_ids):
    mu_endmem_phs = np.dot(elem_comps[ids], mu_elem_local)
    iA, iXmin = phs.affinity_and_comp(T, P, mu_endmem_phs)
    # print(A)
    soln_mu[ids] = iA*iXmin
    
    


In [ ]:
mu[all_soln_endmem_ids]- soln_mu

In [ ]:
mu_bulk_prev = mu_bulk

In [ ]:
%%timeit
# mu_update = np.hstack((soln_mu, pure_mu))

wt_bulk, mu_bulk, ind_assem = min_energy_assemblage(
    bulk_comp, elem_comps, mu_update, TOLmu=10, mu_init=mu_bulk_prev)

In [ ]:
mu_bulk

In [ ]:
len(soln_endmem_ids)

In [ ]:

fun =  lambda mol, mu_endmem_phs=mu_endmem_phs, T=T, P=P: (
    phs.gibbs_energy(T, P, mol=mol)- np.dot(mol,mu_endmem_phs))
jac =  lambda mol, mu_endmem_phs=mu_endmem_phs, T=T, P=P: (
    phs.gibbs_energy(T, P, mol=mol, deriv={'dmol':1})- mu_endmem_phs)

In [ ]:
soln_mu

In [ ]:
mol = np.random.rand(phs.endmember_num)
mol /= mol.sum()


In [ ]:
%%timeit
jac(mol)

In [ ]:
# %%timeit
dmu = phs.chem_potential(T, P, mol=mol)- mu_endmem_phs
dmu

In [ ]:
500*14

In [ ]:


fun =  lambda mol, mu_endmem_phs=mu_endmem_phs, T=T, P=P: (
    phs.gibbs_energy(T, P, mol=mol)- np.dot(mol,mu_endmem_phs))

jac =  lambda mol, mu_endmem_phs=mu_endmem_phs, T=T, P=P: (
    phs.gibbs_energy(T, P, mol=mol, deriv={'dmol':1})- mu_endmem_phs)
hess =  lambda mol, mu_endmem_phs=mu_endmem_phs, T=T, P=P: (
    phs.gibbs_energy(T, P, mol=mol, deriv={'dmol':2}))
Nendmem = phs.endmember_num

# output = opt.minimize(fun, mol0)
bounds = np.tile((0,1),(Nendmem,1) )
# TOL = 1e-5
# constr = opt.LinearConstraint(np.ones(Nendmem),1-TOL,1+TOL)
constr = opt.LinearConstraint(np.ones(Nendmem),1,1)

mol0 = np.random.rand(Nendmem)
mol0 /= np.sum(mol0)
# fun = lambda mol: (mol[0]-.7)**2
fun(mol0)

In [ ]:
mol0 = np.linspace(0,1, 101)
aff = np.zeros(mol0.size)
for ind, imol0 in enumerate(mol0):
    imol = [imol0, 1-imol0]
    aff[ind] = fun(imol)

In [ ]:
plt.figure()
plt.plot(mol0, aff, 'k-')

In [ ]:

mol = np.random.rand(phs.endmember_num)
mol /= mol.sum()
fun(mol)
jac(mol)

In [ ]:
%%timeit

# fun(mol)
jac(mol)

In [ ]:
mu = jac(mol).squeeze()
mu

In [ ]:
opt.minimize(fun, mol, constraints=constr)

In [ ]:
# 500 micro s

In [ ]:
# opt.minimize?

In [ ]:
phs.gibbs_energy(T, P, mol=mol)

In [ ]:
mu_elem_local

In [ ]:
A, Xmin = phs.affinity_and_comp(T, P, mu_endmem_phs)
A

In [ ]:
comps = np.eye(phs.endmember_num)
mol = comps[0]
mol = np.ones(phs.endmember_num)
# mol = np.random.rand(len(mol))
mol /= mol.sum()

G = phs.gibbs_energy(T, P, mol=mol).squeeze()
dG = phs.gibbs_energy(T, P, mol=mol, deriv={'dmol':1}).squeeze()
d2G = phs.gibbs_energy(T, P, mol=mol, deriv={'dmol':2}).squeeze()
Hess = np.linalg.inv(d2G)

In [ ]:
dG-mu_endmem_phs

In [ ]:
opt.minimize?

In [ ]:
opt.minimize(fun, mol)

In [ ]:
d2G

In [ ]:
np.linalg.det(d2G)

In [ ]:
Q = 2*d2G

In [ ]:
np.dot(np.linalg.inv(Q), -(dG-mu_endmem_phs))

In [ ]:
-0.5*np.dot(Hess, dG-mu_endmem_phs)

In [ ]:
-0.5*np.dot(d2G, 1/dG)

In [ ]:
hull_phs_sym = np.unique(phs_sym_uniq[ind_assem])
hull_phs_sym

In [ ]:
# np.dot(wt_bulk,mu_uniq)

In [ ]:
# mu_bulk/1e3

In [ ]:
np.dot(, mu_uniq)

# Determine hull phases

In [ ]:
def get_hull_pts(comps, mu, ind_assem, debug=True, TOL=1e-4, Nelems=Nelems):
    comp_hull = comps[ind_assem,:]
    mu_hull = mu[ind_assem]
    ind_hull = ind_assem.copy()
    
    Nphs = len(mu)
    mask = np.tile(True, Nphs)
    mask[ind_assem] = False
    
    mu_extra = mu[mask]
    comp_extra = comps[mask,:]
    ind_extra = np.arange(Nphs)[mask]
    
    for icomp, imu, ind in zip(comp_extra, mu_extra, ind_extra):
        
        # for ind in inds[Nhull_pts:]:
        iwt_hull, wt_resid = sp.optimize.nnls(comp_hull.T,icomp,maxiter=1000)
        icomp_mod = np.dot(iwt_hull, comp_hull) 
        imu_mod = np.dot(iwt_hull, mu_hull)
        
        icomp_resid = icomp-icomp_mod
        comp_match = np.all(np.abs(icomp_resid)<TOL)
        imaxdev = np.max(np.abs(icomp_resid))
        lower_energy = imu< imu_mod
        
        add_pt = (not comp_match) or (comp_match and lower_energy)
        if debug:
            print(add_pt, '---', len(mu_hull),
                  '(',comp_match,' , ', lower_energy, ') : ', imaxdev)
        if add_pt:
            comp_hull = np.vstack((comp_hull, icomp))
            mu_hull = np.hstack((mu_hull, imu))
            ind_hull = np.hstack((ind_hull, ind))
            
    return comp_hull, mu_hull, ind_hull

In [ ]:
comp_hull, mu_hull, ind_hull = get_hull_pts(
    elem_comps_uniq, mu_uniq, ind_assem, debug=False, TOL=1e-4, Nelems=Nelems)

In [ ]:
comp_hull

In [ ]:
phs_sym_uniq[ind_hull]

In [ ]:
def get_quad_inds(Nelems):
    ind_rows, ind_cols = np.tril_indices(Nelems,-1)
    cross_term_inds = np.vstack((ind_rows,ind_cols))
    return cross_term_inds

In [ ]:
def eval_curv(comps, method, cross_term_inds):
    single_pt = False
    if comps.ndim==1:
        single_pt = True
        comps = comps[np.newaxis,:]
        
    if method=='quad':
        XiXj = comps[:, cross_term_inds[0]]*comps[:, cross_term_inds[1]]
        X2_sum = np.sum(XiXj,axis=1)
        curv_term = X2_sum
    elif method=='quad-full':
        XiXj = comps[:, cross_term_inds[0]]*comps[:, cross_term_inds[1]]
        curv_term = XiXj
    elif method=='xlogx':
        logX = np.log(comps)
        logX[comps==0] = 0
        XlogX = comps*logX
        # XlogX[comps==0] = 0
        XlogX_sum = np.sum(XlogX,axis=1)
        curv_term = XlogX_sum
    else:
        assert False, method + ' is not a valid method for eval_curv.'
        
    if single_pt:
        curv_term = curv_term[0]
    
    return curv_term

In [ ]:
cross_term_inds = get_quad_inds(Nelems)

# Add linear combo points to flesh out hull

In [ ]:
mu_linear = yscl*param[:Nelems]
mu_curv = yscl*param[Nelems:][0]

In [ ]:
mu_curv

In [ ]:
mu_linear

## Build endmembers of pseudo-phase using the coder module

In [ ]:
modelCD = coder.StdStateModel()

In [ ]:
GTP = sym.symbols('GTP')
params = [('GTP','J',GTP)]
modelCD.add_expression_to_model(GTP, params)

In [ ]:
modelCD.set_module_name('pseudo_end')

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
def standardize_formula(form):
    cmp = form.split('O')
    str = ''
    if cmp[0][-1].isdigit():
        str += cmp[0][:-1] + '(' + cmp[0][-1] + ')'
    else:
        str += cmp[0] + '(1)'
    if cmp[1] == '':
        str += 'O'
    else:
        str += 'O(' + cmp[1] + ')'
    return str

In [ ]:
# param

In [ ]:
mu_linear

In [ ]:
sys_elems

In [ ]:
model_type = "calib"
for ind,elm in enumerate(sys_elems):
    imu = mu_linear[ind]
    print(imu)
    if use_oxides_as_basis:
        formula = standardize_formula(elm)
    else:
        formula = elm+'(1)'
    param_dict = {'Phase':elm,'Formula':formula,'T_r':298.15,'P_r':1.0,'GTP':imu}
    print (param_dict)
    result = modelCD.create_code_module(
        phase=param_dict.pop('Phase', None),
        formula=param_dict.pop('Formula', None),
        params=param_dict, module_type=model_type, silent=True)
    print ('Component', elm, 'done!')

Build the code (ignore error messages generated by Cython regarding 'language_level')

In [ ]:
import pseudo_end
%cd ..

In [ ]:
elm_sys=sys_elems

In [ ]:
c = len(elm_sys)
c

In [ ]:
modelCD = coder.SimpleSolnModel(nc=c)

In [ ]:
n = modelCD.n
nT = modelCD.nT
X = n/nT

In [ ]:
len(X)

In [ ]:
# XiXj = np.dot(n,n.T)[cross_term_inds[0], cross_term_inds[1]]/nT**2
# XiXj

In [ ]:
mu = modelCD.mu
mu

In [ ]:
# Tsym = modelCD.get_symbol_for_t()

In [ ]:
G_ss = (n.transpose()*mu)[0]
G_ss

In [ ]:
X.shape

In [ ]:
# dX = X[:,0]- bulk_comp[:,np.newaxis]
# dX

In [ ]:
# X0 = sym.MatrixSymbol('X_0',c,1)
# X0.shape

In [ ]:
# dX = (X[:,0]-X0[:,0])
# dX = X-X0
# dX.shape

In [ ]:
# sym.Matrix(dX)

In [ ]:
# if curv_method=='quad-full':
# curv_string = ''
# quad_strs = []
# for i,j in cross_term_inds.T:
#     # print(i, j)
#     istr = 'k_' + str(i+1) + '_' + str(j+1)
#     curv_string +=  istr + ' '
#     quad_strs.append(istr)
#     
# 
# quad_consts = sym.Matrix(list(sym.symbols(curv_string)))
k_curv = sym.symbols('k_curv')
k_curv

In [ ]:
bulk_string = ''
bulk_strs = []
for i in np.arange(Nelems):
    # print(i, j)
    istr = 'Xbulk_' + str(i+1)
    bulk_string +=  istr + ' '
    bulk_strs.append(istr)
    

bulk_sym = sym.Matrix(list(sym.symbols(bulk_string)))

In [ ]:
bulk_sym

In [ ]:
# quad_consts

In [ ]:
dX = n/nT-bulk_sym
X2_sym = np.dot(dX.T,dX)[0,0]
G_local_quad = nT*k_curv*X2_sym
G_local_quad

In [ ]:
# quad_strs

In [ ]:
# XiXj = np.dot(n,n.T)[cross_term_inds[0], cross_term_inds[1]]/nT**2
# G_quad = nT*np.dot(XiXj, quad_consts)[0]

In [ ]:
# G_quad

In [ ]:
mu_shft = sym.symbols('mu_shft')
mu_shft

In [ ]:
# Ncross_terms = cross_term_inds.shape[1]
# Ncross_terms

In [ ]:
Gshft = mu_shft*nT

In [ ]:
G = G_ss + G_local_quad + Gshft
G

In [ ]:
for istr, isym in zip(bulk_strs, bulk_sym):
    modelCD.add_expression_to_model(
        G, [(istr, 'none', isym)])
    

In [ ]:
modelCD.add_expression_to_model(G, [('k_curv', 'J/m', k_curv)])
modelCD.add_expression_to_model(G, [('mu_shft', 'J/m', mu_shft)])

In [ ]:
modelCD.module = "pseudo_soln"

In [ ]:
# dX = (X[:,0]-bulk_comp[:,np.newaxis])
# dX2 = np.dot(dX.T,dX)[0,0]
# dX2
# 
# G_mix, k_mix = sym.symbols('G_mix k_mix')

In [ ]:
# G_mix = k_mix*dX2
# G_mix

In [ ]:
# G = G_ss + G_mix

In [ ]:
# modelCD.add_expression_to_model(G, [('k_mix', 'none', k_mix)])

In [ ]:
# modelCD.module = "pseudo_soln"

In [ ]:
formula = ''
convert = []
test = []
if use_oxides_as_basis:
    for ind,elm in enumerate(elm_sys):
        ox_index = list(core.chem.oxide_props['oxides']).index(elm)
        ox_cat = core.chem.oxide_props['cations'][ox_index]
        formula += ox_cat + '[' + ox_cat + ']'
        ox_cat_num = core.chem.oxide_props['cat_num'][ox_index]
        if ox_cat_num > 1:
            convert.append('['+str(ind)+']=['+ox_cat+']/'+str(ox_cat_num)+'.0')
        else:
            convert.append('['+str(ind)+']=['+ox_cat+']')
        test.append('['+str(ind)+'] >= 0.0')
    formula += 'O[O]'
else:
    for ind,elm in enumerate(elm_sys):
        formula += elm + '[' + elm + ']'
        convert.append('['+str(ind)+']=['+elm+']')
        test.append('['+str(ind)+'] >= 0.0')
formula, convert, test

In [ ]:
modelCD.formula_string = formula
modelCD.conversion_string = convert
modelCD.test_string = test

In [ ]:
# mu_curv = np.tile(mu_curv/Ncross_terms, Ncross_terms)
mu_curv

In [ ]:
# curv_local
local_curv

In [ ]:
paramValues = {'T_r':298.15,'P_r':1.0}

In [ ]:
# for isym, ival in zip(quad_strs, mu_curv):
#     paramValues[isym] = ival
paramValues['k_curv'] = mu_curv

for ibulk_val, ibulk_str in zip(bulk_comp, bulk_strs):
    paramValues[ibulk_str] = ibulk_val
    
paramValues['mu_shft'] = 0
    
endmembers = []
for elm in elm_sys:
    endmembers.append(str(elm)+'_pseudo_end')

In [ ]:
paramValues

In [ ]:
bulk_comp

In [ ]:
# paramValues = {'k_mix':curv_local,'T_r':298.15,'P_r':1.0,}


In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
# cd ..

In [ ]:
modelCD.create_code_module(
    phase="PseudoPhase", params=paramValues, endmembers=endmembers, 
    prefix="cy", module_type='calib', silent=False)

In [ ]:
pwd

In [ ]:
import pseudo_soln
%cd ..

In [ ]:
endmembers

In [ ]:
G

In [ ]:
#%cd working
#import pseudo_soln
#%cd ..
modelPseudo = model.Database(database="CoderModule", calib="calib", 
                         phase_tuple=('pseudo_soln', {'Psu':['PseudoPhase','solution']}))
Pseudo = modelPseudo.get_phase('Psu')

for phase_name, abbrv in zip(modelPseudo.phase_info.phase_name,modelPseudo.phase_info.abbrev):
    print ('Abbreviation: {0:<10s} Name: {1:<30s}'.format(abbrv, phase_name))

In [ ]:
vals = Pseudo.get_param_values(all_params=True)
names = np.array(Pseudo.param_names)

# Pseudo.set_param_values(param_names=[2],param_values=[1.0])

In [ ]:
vals = Pseudo.get_param_values(all_params=True)
len(vals)

In [ ]:
mudiff = Pseudo.gibbs_energy(T, P, mol=bulk_comp)-mu_bulk

In [ ]:
mu_bulk

In [ ]:
Pseudo.endmember_names

In [ ]:
names

In [ ]:
# ind_kcurv = np.where(names=='k_curv')[0][0]
# k_curv
# Pseudo.set_param_values(param_names=[ind_kcurv], param_values=[0*local_curv[0]])

In [ ]:
np.sum(bulk_comp)

In [ ]:
# mu_bulk
mudiff/2

In [ ]:
# mu_shft_val=+50e3
# mu_shft_val=+10e3
# mu_shft_val=+15e3
# mu_shft_val=+100e3
# mu_shft_val=+1000e3
# mu_shft_val=-mu_bulk+5e3
mu_shft_val= +1169084.0 
Pseudo.set_param_values(param_names=[len(vals)-1], param_values=[mu_shft_val])
Pseudo.get_param_values(all_params=True)

In [ ]:
mudiff = Pseudo.gibbs_energy(T, P, mol=bulk_comp)-mu_bulk
mudiff

In [ ]:
Pseudo.get_param_values(all_params=True)

In [ ]:
# vals[2:] *= 10

In [ ]:
# mu_curv

In [ ]:
# vals

In [ ]:
# for ind, ival in enumerate(vals[2:]):
#     Pseudo.set_param_values(param_names=[ind+2], param_values=[ival])

In [ ]:
# Pseudo.get_param_values(all_params=True)

Check pseudo-phase import by printning some phase characteristics

In [ ]:
print (Pseudo.props['phase_name'])
print (Pseudo.props['formula'])
print (Pseudo.props['molwt'])
print (Pseudo.props['abbrev'])
print (Pseudo.props['endmember_num'])
print (Pseudo.props['endmember_name'])

## Try the equiibrium calculations with the omnicomponent pseudo-phase
#### Choose a phase assemblage

In [ ]:
#stix_phases.keys()
phs_sys  = [Pseudo]

# phs_sys.extend(phases.values())

# phs_sys += [phases['Fsp'], phases['Ol'], phases['Cpx'], phases['Grt']] # solutiopns,
# phs_sys += [phases['Qz'], phases['Ky'], phases['Nph']]
#
#phs_sys  = [Pseudo, stix_phases['Opx']]

In [ ]:
for iphssym in hull_phs_sym:
    phs_sys.append(phases[iphssym])
    
    

In [ ]:
phs_sys

In [ ]:
# phases = get_subsolidus_phases(database=database)

In [ ]:
sys_elems
# phs_sys

In [ ]:
# equil = equilibrate.Equilibrate(['O','Na','Mg','Al','Si','Ca','Fe'], phs_sys)
equil = equilibrate.Equilibrate(sys_elems, phs_sys)

In [ ]:
mu_bulk

In [ ]:
1467815.52

In [ ]:
state = equil.execute(T, P, bulk_comp=bulk_comp, debug=0)
state.print_state()